In [1]:
import hls4ml

2024-04-10 14:46:40.936148: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 14:46:40.941754: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 14:46:41.019260: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 14:46:43.537838: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import torch.quantization
from tqdm import tqdm 
from torch.nn.utils import prune

In [4]:
batch = 1024
num_epochs = 40
layers = [64,32,16]
lr = 0.0005
df = pd.read_parquet('/workspaces/Acc-Higgs-HLS4ML/ML model/df.parquet')
X = pd.read_parquet('/workspaces/Acc-Higgs-HLS4ML/ML model/x.parquet').values  
df['proc'] = df['proc'].apply(lambda x: 1 if x != 0 else 0)
y = df['proc'].values  

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=batch, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch, shuffle=False)


In [5]:
X.shape[1]

140

In [6]:
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.layer0 = nn.Linear(X.shape[1], layers[0])
        self.layer1 = nn.Linear(layers[0], layers[1])
        self.layer2 = nn.Linear(layers[1], layers[2])
        self.layer3 = nn.Linear(layers[2], 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        timings = {}

        start_time = time.time()
        x = torch.relu(self.layer0(x))
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        timings['fc_time'] = time.time() - start_time

        x = self.sigmoid(self.layer3(x))
        
        return x, timings['fc_time']
model = BinaryClassificationModel()

In [21]:
model = BinaryClassificationModel()
model.load_state_dict(torch.load('/workspaces/Acc-Higgs-HLS4ML/ML model/model.pth'))

<All keys matched successfully>

In [25]:
y_torch = model(X_tensor)

In [8]:
import plotting

In [9]:
# convert model to FPGA using hls4ml
config = hls4ml.utils.config.config_from_pytorch_model(model)
print("-----------------------------------")
print("Configuration")
def print_dict(d):
    for key, value in d.items():
        print(f"{key}: {value}")
print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_pytorch_model(
    model, 
    input_shape=(None, 140),
    hls_config=config, 
    output_dir='/workspaces/Acc-Higgs-HLS4ML/HLS4ML/hls4ml_original_model', 
    part='xcu250-figd2104-2L-e'
)

-----------------------------------
Configuration
Model: {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'InputsChannelLast': False, 'TransposeOutputs': True, 'Strategy': 'Latency'}
-----------------------------------
Interpreting Model ...
Topology:
Layer name: layer0, layer type: Dense, input shape: [[None, 140]]
Layer name: relu, layer type: Activation, input shape: [[None, 64]]
Layer name: layer1, layer type: Dense, input shape: [[None, 64]]
Layer name: relu_1, layer type: Activation, input shape: [[None, 32]]
Layer name: layer2, layer type: Dense, input shape: [[None, 32]]
Layer name: relu_2, layer type: Activation, input shape: [[None, 16]]
Layer name: layer3, layer type: Dense, input shape: [[None, 16]]
Layer name: sigmoid, layer type: Activation, input shape: [[None, 1]]
Creating HLS model


In [13]:
import pydot

In [16]:
# Print the model configuration
print(hls_model.config)


In [17]:
# complie and predict
hls_model.compile()
X_test = np.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)

Writing HLS project
Done


In [18]:
from sklearn.metrics import accuracy_score

In [43]:
model.eval()
y_pre = []

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs, _ = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        print(predicted == labels)
        correct += (predicted == labels).sum().item()
        print(correct)
        y_pre.append(predicted)
accuracy0 = correct / total


tensor([[ True, False, False,  ...,  True, False, False],
        [False,  True,  True,  ..., False,  True,  True],
        [False,  True,  True,  ..., False,  True,  True],
        ...,
        [ True, False, False,  ...,  True, False, False],
        [False,  True,  True,  ..., False,  True,  True],
        [False,  True,  True,  ..., False,  True,  True]])
566732
tensor([[ True, False, False,  ...,  True,  True, False],
        [False,  True,  True,  ..., False, False,  True],
        [False,  True,  True,  ..., False, False,  True],
        ...,
        [ True, False, False,  ...,  True,  True, False],
        [ True, False, False,  ...,  True,  True, False],
        [False,  True,  True,  ..., False, False,  True]])
1117398
tensor([[ True, False, False,  ...,  True, False,  True],
        [False,  True,  True,  ..., False,  True, False],
        [False,  True,  True,  ..., False,  True, False],
        ...,
        [ True, False, False,  ...,  True, False,  True],
        [False, 

In [34]:
# compare 
print("Pytorch Accuracy: {}".format(accuracy0))

Pytorch Accuracy: 541.3885
